In [0]:
!pip install networkx
!pip install 

ERROR: You must give at least one requirement to install (see "pip help install")


In [0]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

<h1> Assumption 0</h1>




In [0]:
population_ch = 8.57 * 10**6
population_it = 60.58 * 10**6
population_synthetic=6000 

<h1> Assumption 1</h1>

In [0]:

def p_function(n):
  random_encounter = np.random.random()
  
  return random_encounter/(n-1)
  

def b_subset_of_a(a,b):
  """
  Function to check if all the edges of the previous model where send to the next model
  """
  counter = 0
  big_set = []
  small_set = []
  for i in a:
    big_set.append(tuple(sorted(i)))
  for i in b:
    small_set.append(tuple(sorted(i)))
  if set(small_set).issubset(big_set):
    return True
  return False



def preferential_attachement():
  pass

In [0]:
n = 10
p = 0.1
k = 5

In [0]:
def model(n,p,k):
  # Assumption 1.
  # in is defined by assumption 0, and p*(n-1) is the expected number of connections a person has
  G1 = nx.erdos_renyi_graph(n, p, directed=False)
  # Assumption 1.2
  G2 = nx.stochastic_block_model(sizes, probs, seed=0)
  # Assumption 1.3
  G3 = nx.barabasi_albert_graph(n, k)
  #Get edges from a graph
  #output (u,v) u and v are the nodes, which are connected to each other
  connections = G2.edges()
  G1.add_edges_from(connections)
  return G1


<h1> Assumption 2</h1>


In [0]:
def infection(g, people_infected_perc):
  healthy = 0
  infected = 1
  recovered = 2

  checker = True

  size = len(g)
  people_infected = people_infected_perc * size
  
  #initial infected state
  for i in g.nodes():
    g.nodes[i]['state'] = healthy

  for i in range(people_infected):
    checker = False
    while checker == False:
      seed = np.random.randint(people_infected)
      if g.nodes[seed]['state'] != infected:
        g.nodes[seed]['state'] = infected
        checker = False
      
  #print(g.nodes[seed]['state'])


